In [1]:
#import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#load data

df = pd.read_csv('./DATA/AMES_Final_DF.csv')

In [3]:
#check how the data looks like

df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


In [5]:
#create the X variables for the model

X = df.drop('SalePrice', axis = 1)

In [6]:
#create the y variable for the model

y = df['SalePrice']

In [7]:
#import train-test split library

from sklearn.model_selection import train_test_split

In [8]:
#split the data, the test size will be 10%, and the random state secures the comparabality

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 101)

In [9]:
#standardize the X features due to mass variety of scales and units

from sklearn.preprocessing import StandardScaler

In [10]:
#create an instance

scaler = StandardScaler()

In [11]:
#fit the scaler to X_train only and transform it

scaled_X_train = scaler.fit_transform(X_train)

In [12]:
#transform the X_test data subset

scaled_X_test = scaler.transform(X_test)

In [13]:
#elastic net model uses both lasso and ridge regression and will decide which ratio is best for the data

from sklearn.linear_model import ElasticNet

In [14]:
#create an instance

base_elastic_model = ElasticNet()

In [15]:
#create the param_grid dictionary for the grid search model in order to tune the hyperparameters

param_grid = {'alpha': [0.1, 1, 5, 10, 50, 100], 
              'l1_ratio': [.1, .7, .9, .95, .99, 1]}

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
#run the grid search model

grid_model = GridSearchCV(base_elastic_model,
                         param_grid = param_grid,
                         scoring = 'neg_mean_squared_error',
                         cv = 5,
                         verbose = 1)

In [18]:
#fit it on the train data subset

grid_model.fit(scaled_X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


C:\Users\Public\anaconda3\envs\course\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.394e+11, tolerance: 1.355e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Public\anaconda3\envs\course\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.654e+11, tolerance: 1.308e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Public\anaconda3\envs\course\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

C:\Users\Public\anaconda3\envs\course\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.526e+11, tolerance: 1.438e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Public\anaconda3\envs\course\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.559e+11, tolerance: 1.346e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\Public\anaconda3\envs\course\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [19]:
#check the best lambda and lasso/ridge ratio, this says the best lambda is 100 and the lasso is fully preferred

grid_model.best_params_

{'alpha': 100, 'l1_ratio': 1}

In [20]:
#predict the y variable on the test set

y_pred = grid_model.predict(scaled_X_test)

In [21]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [22]:
#average absolute error between the test data and predicted data

mean_absolute_error(y_test, y_pred)

14195.354900562166

In [23]:
#average squared error between the test data and predicted data

np.sqrt(mean_squared_error(y_test, y_pred))

20558.508566893164